In [ ]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
pd.set_option('mode.chained_assignment',  None)

X = pd.read_csv("data/X_train.csv")
y = pd.read_csv("data/y_train.csv")
X_sub = pd.read_csv("data/X_test.csv")
# print(X.shape,X_sub.shape,y.shape) # (3500, 10) (2482, 10) (3500, 2)

# 사용자 코딩
# print(X.columns) #['cust_id', '총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']
# print(y.columns) # ['cust_id', 'gender']

# 데이터 확인
df = pd.concat([X,X_sub], axis=0, ignore_index=True)
# df.info()	# 결측치 : 환불금액 < 수치형
# print(df.shape) # (5982, 10)

df_o = df[['주구매상품', '주구매지점']]
df_n = df[['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']]
y = y['gender']
# print(df_o.shape, df_n.shape, y.shape) # (5982, 2) (5982, 7) (3500, 1)

# 범주형
# 결측치 - 없음
# 라벨링
from sklearn.preprocessing import LabelEncoder, StandardScaler
df_o.주구매상품 = LabelEncoder().fit_transform(df_o['주구매상품'])	# failed만 안뜨면 괜찮음
df_o.주구매지점 = LabelEncoder().fit_transform(df_o['주구매지점'])
#########################################################################################
# 수치형
# 결측치
# print(df_n.isna().sum()) #환불금액       3906
df_n['환불금액'] = df[df_n['환불금액'].isna()] = df_n['환불금액'].mean()
# print(df_n.isna().sum()) # 0
# 이상치
# ['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']
col = ['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']
# for i in col:
# 	a = i
# 	q1, q3 = df_n[a].quantile([0.25, 0.75])
# 	IQR = q3 - q1
# 	q_lowwer, q_upper = q1-(IQR*1.5), q3+(IQR*1.5)
# 	condition1 = df_n[df_n[a] < q_lowwer]
# 	condition2 = df_n[df_n[a] > q_upper]
# 	print(a, len(condition1)+len(condition2))

# 이상치 대체
for i in col:
	a = i
	q1, q3 = df_n[a].quantile([0.25, 0.75])
	IQR = q3 - q1
	q_lowwer, q_upper = q1-(IQR*1.5), q3+(IQR*1.5)
	condition1 = df_n[df_n[a] < q_lowwer]
	condition2 = df_n[df_n[a] > q_upper]
	df_n.loc[df_n[a] < q_lowwer, a] = q_lowwer
	df_n.loc[df_n[a] > q_upper, a] = q_upper
	
# 이상치 대체 확인
# for i in col:
# 	a = i
# 	q1, q3 = df_n[a].quantile([0.25, 0.75])
# 	IQR = q3 - q1
# 	q_lowwer, q_upper = q1-(IQR*1.5), q3+(IQR*1.5)
# 	condition1 = df_n[df_n[a] < q_lowwer]
# 	condition2 = df_n[df_n[a] > q_upper]
# 	print(a, len(condition1)+len(condition2))

# 스케일링
df_n = pd.DataFrame(StandardScaler().fit_transform(df_n)) # 스케일링은 한번에 해도 됨
# print(df_n.shape, df_o.shape)

# 홀드아웃
df = pd.concat([df_n, df_o], axis=1)
X = df.iloc[:len(X),:]
X_sub = df.iloc[len(X):,:]
# print(X.shape, X_sub.shape, y.shape) #(3500, 9) (2482, 9) (3500, 1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
																									 random_state=777)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) #(2450, 9) (1050, 9) (2450,) (1050,)

# 모델링
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
print(model.score(X_train,y_train), model.score(X_test,y_test))

from sklearn.metrics import roc_auc_score
pred_proba = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, pred_proba)
print(roc_auc)

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
sub = pd.read_csv("data/X_test.csv")
pred_proba = model.predict_proba(X_sub)[:,1]
pd.DataFrame({'cust_id': sub.cust_id, 'gender': pred_proba}).to_csv('8585.csv', index=False)

df2 = pd.read_csv('8585.csv')
print(df2)
